# Agent Development Notebook

Use `sk_agent` module to develop Agents.

## Setup and Imports

Import from our new modular structure instead of defining everything inline.

In [1]:
import asyncio
import json
import os
import sys
from pathlib import Path
from dotenv import load_dotenv
from IPython.display import display, Markdown

# Add root directory to Python path
sys.path.append("..")    



# Core Semantic Kernel imports
from semantic_kernel import Kernel
from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.connectors.ai.open_ai import AzureChatPromptExecutionSettings
from semantic_kernel.contents import ChatMessageContent, FunctionCallContent, FunctionResultContent
from semantic_kernel.contents.utils.author_role import AuthorRole
from semantic_kernel.contents.chat_history import ChatHistory
from semantic_kernel.functions import KernelArguments

# Group Chat Orchestration imports
from semantic_kernel.agents import GroupChatOrchestration
from semantic_kernel.agents.runtime import InProcessRuntime

# Import from our modular structure
from sk_agents.services import get_reasoning_service, get_chat_service
from sk_agents.orchestration.managers import SingleAgentGroupChatManager, DiscoveryAgentGroupChatManager
from sk_agents.agents import create_agent, load_agent_from_config
from plugins.file_system import FileSystemPlugin
from plugins.discovery_memory import DiscoveryMemoryPlugin
from sk_agents.config import MAX_ROUNDS

# Load environment variables
load_dotenv()

print("✅ All imports loaded successfully!")

✅ All imports loaded successfully!


## Configure Services

Use new service configuration functions.

In [2]:
# Configure services using new functions
reasoning_service = get_reasoning_service(reasoning_effort="high")
chat_service = get_chat_service()

print("✅ Services configured successfully!")
print(f"Reasoning service: {reasoning_service.service_id}")
print(f"Chat service: {chat_service.service_id}")

✅ Services configured successfully!
Reasoning service: reasoning
Chat service: chat


## Initialize FileSystemPlugin

Set up with consult/ directory as in original notebook.

In [3]:
# Initialize FileSystemPlugin with consult/ as base directory
consult_path = Path("../consult copy").resolve()
print(f"📁 Setting FileSystemPlugin base path to: {consult_path}")

file_system_plugin = FileSystemPlugin(base_path=str(consult_path))

# Verify the directory exists
if not consult_path.exists():
    raise ValueError(f"❌ Directory {consult_path} does not exist!")
    
print(f"✅ FileSystemPlugin initialized with base path: {consult_path}")

📁 Setting FileSystemPlugin base path to: /home/agangwal/lseg-migration-agent/migration-agent/consult copy
✅ FileSystemPlugin initialized with base path: /home/agangwal/lseg-migration-agent/migration-agent/consult copy


## Create Agent

Test both our helper function and config loading approaches.

In [4]:
### NOTE: If cache exists, it will be re-used.
memory_plugin = DiscoveryMemoryPlugin(repos_path=consult_path / "repos")

In [5]:
response = await memory_plugin.get_all_repos()

[1/10] (10.0%) Analyzing: frontend
[2/10] (20.0%) Analyzing: docs
[3/10] (30.0%) Analyzing: infrastructure
[4/10] (40.0%) Analyzing: lambda
[5/10] (50.0%) Analyzing: migration_tests
[6/10] (60.0%) Analyzing: pipeline-mapping
[7/10] (70.0%) Analyzing: pipeline-sign-off
[8/10] (80.0%) Analyzing: legacy-frontend
[9/10] (90.0%) Analyzing: tests
[10/10] (100.0%) Analyzing: consultation_analyser


In [6]:
response

{'success': True,
 'data': {'frontend': {'name': 'frontend',
   'path': 'frontend',
   'file_extensions': {'.json': 2, '.js': 1, '': 1, '.md': 1},
   'frameworks': ['Express.js'],
   'total_files': 5,
   'total_lines': 1251,
   'has_readme': True,
   'assigned_components': [],
   'discovery_status': 'No insights added. Assigned to no components.'},
  'docs': {'name': 'docs',
   'path': 'docs',
   'file_extensions': {'.md': 6, '.png': 1},
   'frameworks': [],
   'total_files': 7,
   'total_lines': 1745,
   'has_readme': False,
   'assigned_components': [],
   'discovery_status': 'No insights added. Assigned to no components.'},
  'infrastructure': {'name': 'infrastructure',
   'path': 'infrastructure',
   'file_extensions': {'.tf': 18, '.md': 1, '.png': 1, '.sh': 1},
   'frameworks': ['Terraform'],
   'total_files': 21,
   'total_lines': 4287,
   'has_readme': True,
   'assigned_components': [],
   'discovery_status': 'No insights added. Assigned to no components.'},
  'lambda': {'name'

In [7]:
await memory_plugin.get_unanalyzed_repos()

{'success': True,
 'data': {'frontend': {'name': 'frontend',
   'path': 'frontend',
   'file_extensions': {'.json': 2, '.js': 1, '': 1, '.md': 1},
   'frameworks': ['Express.js'],
   'total_files': 5,
   'total_lines': 1251,
   'has_readme': True,
   'discovery_status': 'No insights added. Assigned to no components.',
   'suggested_investigation': ['Read README.md to understand stated purpose',
    'Examine config files: package.json, Dockerfile',
    'Check package.json and look for index.js or server.js']},
  'docs': {'name': 'docs',
   'path': 'docs',
   'file_extensions': {'.md': 6, '.png': 1},
   'frameworks': [],
   'total_files': 7,
   'total_lines': 1745,
   'has_readme': False,
   'discovery_status': 'No insights added. Assigned to no components.',
   'suggested_investigation': ['Read any documentation files to understand purpose']},
  'infrastructure': {'name': 'infrastructure',
   'path': 'infrastructure',
   'file_extensions': {'.tf': 18, '.md': 1, '.png': 1, '.sh': 1},
   

In [8]:
print((await memory_plugin.generate_discovery_report())['data']['report'])

# Legacy Application Discovery Report

**Generated:** 2025-08-13 00:35:30
**Base Path:** `/home/agangwal/lseg-migration-agent/migration-agent/consult copy/repos`

## Executive Summary

- **Total Repositories:** 10
- **Investigation Progress:** 0.0%
- **Repositories with Insights:** 0
- **Logical Components:** 0
- **Unassigned Repositories:** 10

## Repository Inventory

### Needs Investigation (10)

**consultation_analyser**
- File types: .py: 129, .mjs: 64, .js: 48, .html: 38, .json: 7
- Files: 287
- Lines: 20,273
- Status: No insights added. Assigned to no components.
- Components: Unassigned

**docs**
- File types: .md: 6, .png: 1
- Files: 7
- Lines: 1,745
- Status: No insights added. Assigned to no components.
- Components: Unassigned

**frontend**
- Frameworks: Express.js
- File types: .json: 2, .js: 1, : 1, .md: 1
- Files: 5
- Lines: 1,251
- Status: No insights added. Assigned to no components.
- Components: Unassigned

**infrastructure**
- Frameworks: Terraform
- File types: .tf

In [9]:
await memory_plugin.get_components_summary()

{'success': True,
 'data': {'components': {},
  'validation_results': {'unassigned_repos': ['frontend',
    'docs',
    'infrastructure',
    'lambda',
    'migration_tests',
    'pipeline-mapping',
    'pipeline-sign-off',
    'legacy-frontend',
    'tests',
    'consultation_analyser'],
   'multi_assigned_repos': [],
   'orphaned_components': [],
   'assignment_coverage': 0.0}},
 'error': None,
 'suggestions': ['No components created yet',
  'Use add_component() to create logical groupings for repositories',
  'Start with clear business functions or technology stacks'],
 'metadata': {'total_components': 0,
  'total_repositories': 10,
  'assignment_coverage': 0.0,
  'unassigned_count': 10}}

In [10]:
# Method 1: Example - Using create_agent helper with explicit parameters
# analysis_agent_direct = create_agent(
#     name="CodebaseAnalysisAndTestingAgent",
#     service=reasoning_service,
#     description="Code analysis agent with dual objectives: analyze codebase and test FileSystemPlugin tools.",
#     instructions="""You are a helpful assistant.

# """,
#     plugins=[file_system_plugin]
# )

# print(f"✅ Agent created directly: {analysis_agent_direct.name}")

### Preferred Method: Using config-based agent creation
analysis_agent_config = load_agent_from_config(
    "discovery_agent",
    service=reasoning_service,
    plugins=[file_system_plugin, memory_plugin],
)
print(f"✅ Agent created from config: {analysis_agent_config.name}")

# Use the config-based agent for the test
analysis_agent = analysis_agent_config

✅ Agent created from config: LegacyApplicationDiscoveryAgent


## Setup Response Tracking

Same callback and tracking as original notebook.

In [ ]:
# Messages tracking (same as original)
MESSAGES = []

async def agent_response_callback(message: ChatMessageContent) -> None:
    """Display agent responses with function call details."""
    print(f"\n{'='*60}")
    print(f"📝 {message.name}: {message.role}")
    print(f"{'='*60}")
    
    MESSAGES.append(message.model_dump())
    
    # Display message content
    if message.content:
        print(f"\n💭 AGENT REASONING:")
        print(message.content)
    
    # Display function calls and results
    for item in message.items or []:
        if isinstance(item, FunctionCallContent):
            print(f"\n🔧 FUNCTION CALL: {item.name}")
            print(f"📥 Arguments: {json.dumps(item.arguments, indent=2)}")
            
        elif isinstance(item, FunctionResultContent):
            print(f"\n📤 FUNCTION RESULT:")
            try:
                # Try to parse and prettify JSON result
                result_data = json.loads(item.result) if isinstance(item.result, str) else item.result
                print(json.dumps(result_data, indent=2))
            except (json.JSONDecodeError, TypeError):
                # If not JSON, display as string
                print(str(item.result))

print("✅ Response callback configured!")

✅ Response callback configured!


## Create Group Chat Orchestration

Use our extracted SingleAgentGroupChatManager.

In [18]:
manager = DiscoveryAgentGroupChatManager( # Imported from the module. Reduce verbosity.
        topic="Analyze code repositories in a legacy system. Create insights for each repo. Create logical components and assign repos to those components. Ensure each repo assigned to at least one component.",
        service=chat_service,
        max_rounds=MAX_ROUNDS,  # Use config value
    )

In [ ]:
# Create group chat orchestration using our extracted manager
group_chat = GroupChatOrchestration(
    members=[analysis_agent],
    manager=manager,
    agent_response_callback=agent_response_callback,
)

print("✅ Group chat orchestration created with extracted SingleAgentGroupChatManager!")
print(f"🔄 Max rounds: {MAX_ROUNDS}")

✅ Group chat orchestration created with extracted SingleAgentGroupChatManager!
🔄 Max rounds: 20


## Execute Agent Analysis

Run the same task as the original notebook.

In [13]:
# Define the same task as original notebook

# Considering a general prompt catalog in sk_agents module down the line as we starndardize

agent_task = """Analyze code repositories in a legacy system. Create insights for each repo. Create logical components and assign repos to those components. Ensure each repo assigned to at least one component.

Ensure you use the memory tools to track progress as you progress through the analysis. 

At the very end, use get_unanalyzed_repos() to confirm all repos have been analyzed. You can see your complete work in get_all_repos() and fill in any last minute insights.

Start immediately and don't stop until complete."""

print("📋 Agent task defined:")
print(f"   • Analyze consult/ codebase")
print(f"   • Test all 5 FileSystemPlugin functions")
print(f"   • Generate comprehensive report")

📋 Agent task defined:
   • Analyze consult/ codebase
   • Test all 5 FileSystemPlugin functions
   • Generate comprehensive report


In [14]:
# Execute the agent using group chat orchestration
print("🚀 Starting AI agent analysis and testing...")
print(f"🎯 Task: {agent_task[:100]}...")
print("\n" + "="*80)
print("AGENT EXECUTION LOG")
print("="*80)

# Create runtime for orchestration
runtime = InProcessRuntime()
runtime.start()

try:
    # Invoke the group chat orchestration
    orchestration_result = await group_chat.invoke(
        task=agent_task,
        runtime=runtime
    )
    
    # Get the final result
    final_response = await orchestration_result.get(timeout=1200)  # 20 minute timeout

    print("\n" + "="*80)
    print("🎉 AGENT EXECUTION COMPLETED")
    print("="*80)
    
    if final_response:
        print(f"\n✅ Final response received")
        print(f"📊 Response length: {len(final_response.content) if hasattr(final_response, 'content') else len(str(final_response))} characters")
    else:
        print("❌ No final response received")
        
finally:
    # Save messages for comparison
    with open("../migration_test_responses.json", "w") as f:
        json.dump(MESSAGES, f, indent=2)
    await runtime.stop_when_idle()

🚀 Starting AI agent analysis and testing...
🎯 Task: Analyze code repositories in a legacy system. Create insights for each repo. Create logical componen...

AGENT EXECUTION LOG

🤖 Termination Check - Should terminate: False
📝 Reason: The agent has not yet demonstrated completion of the following steps: (1) running get_unanalyzed_repos() at the very end to confirm all repositories have been analyzed, (2) confirming that every repo is assigned to at least one logical component, and (3) consolidating and presenting the final report. Therefore, not all objectives are verified as complete.


📝 LegacyApplicationDiscoveryAgent: AuthorRole.ASSISTANT

🔧 FUNCTION CALL: DiscoveryMemoryPlugin-get_all_repos
📥 Arguments: "{}"

📝 LegacyApplicationDiscoveryAgent: AuthorRole.TOOL

📤 FUNCTION RESULT:
{
  "success": true,
  "data": {
    "frontend": {
      "name": "frontend",
      "path": "frontend",
      "file_extensions": {
        ".json": 2,
        ".js": 1,
        "": 1,
        ".md": 1
      

In [15]:
print((await memory_plugin.generate_discovery_report())['data']['report'])

# Legacy Application Discovery Report

**Generated:** 2025-08-13 00:49:15
**Base Path:** `/home/agangwal/lseg-migration-agent/migration-agent/consult copy/repos`

## Executive Summary

- **Total Repositories:** 10
- **Investigation Progress:** 100.0%
- **Repositories with Insights:** 10
- **Logical Components:** 3
- **Unassigned Repositories:** 0

## Repository Inventory

### Complete (Has Insights & Assigned) (10)

**consultation_analyser**
- File types: .py: 129, .mjs: 64, .js: 48, .html: 38, .json: 7
- Files: 287
- Lines: 20,273
- Status: Insights added. Assigned to components: consultation-service.
- Components: consultation-service
- **Insights:**
  - purpose: Django-based monolithic web application for consultation management, providing REST APIs and web UI.
  - business_function: Manage creation, processing, and analysis of consultations
  - architecture: Python Django monolith with Jinja2 templates, PostgreSQL database, migrations, ASGI/WSGI entrypoints
  - key_dependencies: ['

## Display Final Report

Show the agent's final report in formatted markdown.

In [16]:
if final_response:
    print("📋 RENDERING AGENT REPORT")
    print("="*50)
    
    # Extract the content based on the response type
    report_content = None
    
    if isinstance(final_response, ChatMessageContent):
        report_content = final_response.content
    elif isinstance(final_response, str):
        report_content = final_response
    elif hasattr(final_response, 'content'):
        report_content = final_response.content
    
    if report_content:
        # Display the final report as formatted markdown
        display(Markdown(report_content))
        
        # Also save to file for comparison
        with open("../../migration_test_report.md", "w") as f:
            f.write(report_content)
        print(f"\n💾 Saved report to migration_test_report.md")
        
    else:
        print("⚠️ Could not extract report content from response")
        print(f"Response type: {type(final_response)}")
        print(f"Response: {str(final_response)[:500]}...")
else:
    print("❌ No final report to display")

📋 RENDERING AGENT REPORT


All repositories analyzed and assigned. Discovery complete. You can verify via get_all_repos() and produce the final report with generate_discovery_report().


💾 Saved report to migration_test_report.md


## Execution Summary

Compare with original notebook performance.

In [17]:
# Provide execution summary
print("📊 MIGRATION TEST SUMMARY")
print("="*40)
print(f"🤖 Agent: {analysis_agent.name}")
print(f"🧠 Reasoning Service: {reasoning_service.service_id}")
print(f"💬 Chat Service: {chat_service.service_id}")
print(f"📁 Base Directory: {consult_path}")
print(f"🔄 Manager: SingleAgentGroupChatManager (extracted)")
print(f"⚙️ Max Rounds: {MAX_ROUNDS}")

if final_response:
    if isinstance(final_response, ChatMessageContent):
        print(f"📝 Final Report Length: {len(final_response.content)} characters")
    elif isinstance(final_response, str):
        print(f"📝 Final Report Length: {len(final_response)} characters")
    else:
        print(f"📝 Final Response Type: {type(final_response)}")

print(f"\n📈 Total Messages Captured: {len(MESSAGES)}")
print(f"💾 Messages saved to: migration_test_responses.json")



📊 MIGRATION TEST SUMMARY
🤖 Agent: LegacyApplicationDiscoveryAgent
🧠 Reasoning Service: reasoning
💬 Chat Service: chat
📁 Base Directory: /home/agangwal/lseg-migration-agent/migration-agent/consult copy
🔄 Manager: SingleAgentGroupChatManager (extracted)
⚙️ Max Rounds: 20
📝 Final Report Length: 157 characters

📈 Total Messages Captured: 142
💾 Messages saved to: migration_test_responses.json
